# Modlling for engagement 

now we have the features and the lables, we are ready for modelling 
- Classification vs regression
- 9 vs 5 vs 3 scales lables 
- 2.5 vs 5 s window (as is features, average featuers and concatenate features)
- Two-stream Fusion on RGB + Flow 

This should be done over:
- different network artchictures
- cross different familys cross-validation 

Later on:
- Handcrafted features 

### imports and vars

In [1]:
import sys
import numpy as np
import os
import pandas as pd
import itertools
import pickle

from collections import Counter
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler as resample

lables_path = './labels/'
features_path = './features/'
results_path = './modelling_results/'

### Keras modeling class 

### help functions 


In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker
from hpbandster.optimizers import BOHB as BOHB
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

import tensorflow_addons as tfa

import logging
logging.basicConfig(level=logging.ERROR)

class KerasWorker(Worker):
    def __init__(self, input_shape, output_shape, problemType,
                 x_train, y_train, x_validation, y_validation,
                 x_test, y_test, shared_directory, **kwargs):
            super().__init__(**kwargs)
            self.input_shape = (input_shape, )
            self.num_classes = output_shape
            self.batch_size = 64
            self.save_dic = shared_directory
            
            self.problemType = problemType

            self.x_train, self.y_train = x_train, y_train
            self.x_validation, self.y_validation = x_validation, y_validation
            self.x_test, self.y_test = x_test, y_test

    def compute(self, config, budget, working_directory, *args, **kwargs):
            model = Sequential()
            model.add(Dense(units=config['start_neurons_units'],
                            # activation=config['start_neurons_activation'],
                            activation='relu',
                            input_shape=self.input_shape))


            if config['num_dense_layers'] > 1:
                model.add(Dense(units=config['dense1_units'],
                                # activation=config['dense1_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout1_rate']))

            if config['num_dense_layers'] > 2:
                model.add(Dense(units=config['dense2_units'],
                                # activation=config['dense2_activation'],
                                activation='relu',
                                input_shape=self.input_shape))
                model.add(Dropout(config['dropout2_rate']))

            model.add(Dense(self.num_classes, activation='softmax'))


            if config['optimizer'] == 'Adam':
                    optimizer = tf.keras.optimizers.Adam(lr=config['lr'])
            else:
                    optimizer = tf.keras.optimizers.SGD(lr=config['lr'], momentum=config['sgd_momentum'])
            
            loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
            
            METRICS = [
                  tf.keras.metrics.TruePositives(name='tp'),
                  tf.keras.metrics.FalsePositives(name='fp'),
                  tf.keras.metrics.TrueNegatives(name='tn'),
                  tf.keras.metrics.FalseNegatives(name='fn'),
                  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                  tf.keras.metrics.Precision(name='precision'),
                  tf.keras.metrics.Recall(name='recall'),
                  tf.keras.metrics.AUC(name='auc'),
                  tf.keras.metrics.BinaryCrossentropy(name='BinaryCrossentropy'),
                  tfa.metrics.MatthewsCorrelationCoefficient(num_classes=self.num_classes)
            ]

            #METRICS.append(tfa.metrics.MatthewsCorrelationCoefficient(num_classes=self.num_classes))
            
            the_columns = ['thisloss','tp','fp','tn','fn','acc','prec','rec','auc','BC','MCC']
            
            model.compile(
                loss=loss_fn,
                optimizer=optimizer,
                metrics=METRICS
            )

            # model.summary()
            _history = model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=int(budget),
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            val_acc_per_epoch = _history.history['val_accuracy']
            best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

            model.fit(self.x_train, self.y_train,
                              batch_size=self.batch_size,
                              epochs=best_epoch,
                              verbose=0,
                              validation_data=(self.x_validation, self.y_validation))

            train_score = model.evaluate(self.x_train, self.y_train, verbose=0)
            val_score = model.evaluate(self.x_validation, self.y_validation, verbose=0)
            test_score = model.evaluate(self.x_test, self.y_test, verbose=0)
            
            #print(test_score)

            resultsDF = pd.DataFrame([train_score,val_score,test_score],
                                     columns=the_columns,
                                     index=["train_score", "val_score", "test_score"],)
            # print(resultsDF)
            test_predictions_baseline = model.predict(self.x_test)
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_true.out'), self.y_test, delimiter=',')
            np.savetxt(os.path.join(self.save_dic,'testing_finalResults_pred.out'), test_predictions_baseline, delimiter=',')

            return ({
                'loss': test_score[0],  
                'info':  resultsDF.to_dict('index')
            })
        
#             #import IPython; IPython.embed()
#             return ({
#                 'loss': 1-val_score[1], # remember: HpBandSter always minimizes!
#                 'info': {'test accuracy': test_score[1],
#                             'train accuracy': train_score[1],
#                             'validation accuracy': val_score[1],
#                             'number of parameters': model.count_params(),
#                         }

#             })


    @staticmethod
    def get_configspace():
            """
            It builds the configuration space with the needed hyperparameters.
            It is easily possible to implement different types of hyperparameters.
            Beside float-hyperparameters on a log scale, it is also able to handle categorical input parameter.
            :return: ConfigurationsSpace-Object
            """
            cs = CS.ConfigurationSpace()

            lr = CSH.UniformFloatHyperparameter('lr', lower=1e-6, upper=1e-1, default_value='1e-2', log=True)

            # For demonstration purposes, we add different optimizers as categorical hyperparameters.
            # To show how to use conditional hyperparameters with ConfigSpace, we'll add the optimizers 'Adam' and 'SGD'.
            # SGD has a different parameter 'momentum'.
            optimizer = CSH.CategoricalHyperparameter('optimizer', ['Adam', 'SGD'])

            sgd_momentum = CSH.UniformFloatHyperparameter('sgd_momentum', lower=0.0, upper=0.99, default_value=0.9, log=False)

            cs.add_hyperparameters([lr, optimizer, sgd_momentum])



            num_dense_layers =  CSH.UniformIntegerHyperparameter('num_dense_layers', lower=1, upper=3, default_value=2)

            start_neurons_units = CSH.UniformIntegerHyperparameter('start_neurons_units', lower=32, upper=512, default_value=32, log=True)
            dense1_units = CSH.UniformIntegerHyperparameter('dense1_units', lower=8, upper=128, default_value=16, log=True)
            dense2_units = CSH.UniformIntegerHyperparameter('dense2_units', lower=4, upper=64, default_value=8, log=True)

            cs.add_hyperparameters([num_dense_layers, start_neurons_units, dense1_units, dense2_units])

            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu', 'tanh', 'sigmoid'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu', 'tanh', 'sigmoid'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu', 'tanh', 'sigmoid'])
            # start_neurons_activation = CSH.CategoricalHyperparameter('start_neurons_activation', ['relu'])
            # dense1_activation = CSH.CategoricalHyperparameter('dense1_activation', ['relu'])
            # dense2_activation = CSH.CategoricalHyperparameter('dense2_activation', ['relu'])
            #
            # cs.add_hyperparameters([start_neurons_activation, dense1_activation, dense2_activation])

            dropout1_rate = CSH.UniformFloatHyperparameter('dropout1_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)
            dropout2_rate = CSH.UniformFloatHyperparameter('dropout2_rate', lower=0.0, upper=0.9, default_value=0.5, log=False)

            cs.add_hyperparameters([dropout1_rate, dropout2_rate])


            # The hyperparameter sgd_momentum will be used,if the configuration
            # contains 'SGD' as optimizer.
            cond = CS.EqualsCondition(sgd_momentum, optimizer, 'SGD')
            cs.add_condition(cond)

            # You can also use inequality conditions:
            cond = CS.GreaterThanCondition(dense1_units, num_dense_layers, 1)
            cs.add_condition(cond)

            cond = CS.GreaterThanCondition(dense2_units, num_dense_layers, 2)
            cs.add_condition(cond)

            return cs


In [4]:
def load_sub_dataset_twostream(familiesSet, label_folder, feature_folder):
    # append all rows of subjects, and their lables
    allFrames = np.array([])
    allLables = np.array([])
    
    rgb_feature_folder = feature_folder.replace(featureType,'rgb')
    flow_feature_folder = feature_folder.replace(featureType,'flow')
    for this_family in familiesSet:
        onlyfiles = [f for f in os.listdir(rgb_feature_folder) if
                       os.path.isfile(os.path.join(rgb_feature_folder, f))
                       and f.startswith(this_family + '_')]
        onlyfiles.sort()

        for this_file in onlyfiles:
            this_lable_file = this_file.replace('_'+this_file.split('_')[4],'')+'.npy'
            currLabel = np.load(os.path.join(label_folder,this_lable_file))
            
            rgb_currData = np.load(os.path.join(rgb_feature_folder,this_file))    
            flow_currData = np.load(os.path.join(flow_feature_folder,this_file.replace('rgb','flow')))
            
            currData = np.hstack((rgb_currData, flow_currData))
            
            if allFrames.shape[0] ==0:
                allFrames = currData
                allLables = currLabel
            else:
                allFrames = np.vstack((allFrames, currData))
                allLables = np.hstack((allLables, currLabel))

    return allFrames, allLables

def load_sub_dataset(familiesSet, label_folder, feature_folder, featureType):
    # append all rows of subjects, and their lables
    allFrames = np.array([])
    allLables = np.array([])
    
    if featureType == 'twostream':
        return load_sub_dataset_twostream(familiesSet, label_folder, feature_folder)
    
    for this_family in familiesSet:
        # F10_Interaction_1_P27_rgb.npy
        onlyfiles = [f for f in os.listdir(feature_folder) if
                       os.path.isfile(os.path.join(feature_folder, f))
                       and f.startswith(this_family + '_')]
        onlyfiles.sort()

        for this_file in onlyfiles:
            currData = np.load(os.path.join(feature_folder,this_file))            
            substr = '_'.join(this_file.split('_')[4:])
            this_lable_file = this_file.replace('_'+substr,'')+'.npy'
            currLabel = np.load(os.path.join(label_folder,this_lable_file))
            cutoff = min(len(currData), len(currLabel))
            currData = currData[:cutoff,:]
            currLabel =currLabel[:cutoff]
            
            
            if allFrames.shape[0] ==0:
                allFrames = currData
                allLables = currLabel
            else:
                allFrames = np.vstack((allFrames, currData))
                allLables = np.hstack((allLables, currLabel))

    return allFrames, allLables

def load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, label_folder, feature_folder,\
                              prblemType, featureType, num_classes):
    #simple sampeling method
    #TODO: SMOTE, DeepSMOTE, DeepFake?
    sm = resample()
    # load all train
    trainX, trainy = load_sub_dataset(trainSubjs, label_folder, feature_folder, featureType)
    trainX, trainy = shuffle(trainX, trainy)
    trainX, trainy = sm.fit_resample(trainX, trainy)
    print(trainX.shape, Counter(trainy))
    # train_class_weight = sumary_data(trainy)

    # load validation
    valX, valy = load_sub_dataset(valSubjs, label_folder, feature_folder, featureType)
    valX, valy = shuffle(valX, valy)
    valX, valy = sm.fit_resample(valX, valy)
    print(valX.shape, Counter(valy))
    # val_class_weight = sumary_data(valy)


    # load all test
    testX, testy = load_sub_dataset(testSubjs, label_folder, feature_folder, featureType)
    testX, testy = sm.fit_resample(testX, testy)
    print(testX.shape, Counter(testy))
    # test_class_weight = sumary_data(testy)

    # one hot encode y
    trainy = tf.keras.utils.to_categorical(trainy,  num_classes=num_classes)
    valy = tf.keras.utils.to_categorical(valy,  num_classes=num_classes)
    testy = tf.keras.utils.to_categorical(testy,  num_classes=num_classes)
    
    return trainX, trainy, valX, valy , testX, testy, num_classes
    
def create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, num_classes):    
    # repeat experiment
    temp = {}
    all_trainSubjs = [['F' + str(i) for i in [1, 2, 3, 4, 5, 6, 8]]]
    all_valSubjs = [['F' + str(i) for i in [11, 17]]]
    all_testSubjs = [['F' + str(i) for i in [7, 10, 13]]]
    
    min_budget = 9
    max_budget = 243
    n_iterations = 50
    num_workers = 12

    for r in range(len(all_trainSubjs)):
        shared_directory = result_folder + '_'+ str(r) 
        if os.path.exists(shared_directory):
            print(shared_directory,' already processed')
            continue
        
        print(shared_directory,' under processing')
        classType = os.path.basename(shared_directory)

        host = hpns.nic_name_to_host('lo')
        result_logger = hpres.json_result_logger(directory=shared_directory, overwrite=True)
        NS = hpns.NameServer(run_id=classType, host=host, port=0, working_directory=shared_directory)
        ns_host, ns_port = NS.start()
    
        # load data
        trainSubjs = all_trainSubjs[r]
        valSubjs = all_valSubjs[r]
        testSubjs = all_testSubjs[r]
        
        trainX, trainy, valX, valy, testX, testy, num_classes = \
            load_dataset_selectedSubj(trainSubjs, valSubjs, testSubjs, \
                                      label_folder, feature_folder, prblemType, featureType, num_classes)
        
        
        n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], num_classes        
        
        workers = []
        for i in range(num_workers):
            worker = KerasWorker(n_features, n_outputs, prblemType, \
                                 trainX, trainy, valX, valy, testX, testy, \
                                 shared_directory,
                                 run_id=classType,host=host, nameserver=ns_host, nameserver_port=ns_port,
                                 id=i)
            worker.run(background=True)
            workers.append(worker)

        bohb = BOHB(configspace=worker.get_configspace(),
                  run_id=classType,
                  host=host,
                  nameserver=ns_host,
                  nameserver_port=ns_port,
                  result_logger=result_logger,
                  min_budget=min_budget, max_budget=max_budget
                    )
        res = bohb.run(n_iterations=1,  min_n_workers=num_workers)

        id2config = res.get_id2config_mapping()
        incumbent = res.get_incumbent_id()

        print('Best found configuration:', id2config[incumbent]['config'])
        # print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
        # print('A total of %i runs where executed.' % len(res.get_all_runs()))
        # print('Total budget corresponds to %.1f full function evaluations.' % (
        #             sum([r.budget for r in res.get_all_runs()]) / max_budget))

        # store results
        with open(os.path.join(shared_directory, 'results.pkl'), 'wb') as fh:
            pickle.dump(res, fh)

        # shutdown
        bohb.shutdown(shutdown_workers=True)
        NS.shutdown()


### modeling

In [4]:
prblemTypes = ['classification']
featureTypes = ['rgb','flow', 'twostream']
classes = [9,5,3]
# divides = [2.5, 5]
fusionTypes = ['conc','avg','none']

permutations=[ prblemTypes, featureTypes, classes, fusionTypes]
all_permutations = list(itertools.product(*permutations))
print(len(all_permutations))
for this_permutation in all_permutations:
    (prblemType, featureType, eng_lvls, fusionType) = this_permutation
    classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
    divide = 2.5 if fusionType == 'none' else 5
    
    print('Working on: ',prblemType, featureType, eng_lvls, divide, fusionType)
    
    label_folder = os.path.join(lables_path,'_'.join([classType,'eng_lvl',prblemType,str(eng_lvls),str(divide)]))

    
    if divide == 5:
        extra_txt = '_'.join([fusionType,str(divide)]) 
        feature_folder = os.path.join(features_path,'_'.join(['i3d',featureType,'features',extra_txt]))
    else:
        feature_folder = os.path.join(features_path,'_'.join(['i3d',featureType,'features']))
        
    
    #save results like: rgb_classification_9_2.5_none
    result_folder = os.path.join(results_path,'_'.join([featureType,prblemType,str(eng_lvls),str(divide),fusionType]))

    create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, eng_lvls)#, divide,fusionType)
    

27
Working on:  classification rgb 9 5 conc
./modelling_results/rgb_classification_9_5_conc_0  under processing
(32410, 2048) Counter({2.0: 4630, 3.0: 4630, 1.0: 4630, 4.0: 4630, 0.0: 4630, -2.0: 4630, -1.0: 4630})
(7875, 2048) Counter({3.0: 1125, 2.0: 1125, 0.0: 1125, 1.0: 1125, 4.0: 1125, -2.0: 1125, -1.0: 1125})
(4790, 2048) Counter({0.0: 958, 1.0: 958, 2.0: 958, 3.0: 958, 4.0: 958})


2021-10-27 23:23:12.133007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-27 23:23:12.138034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-10-27 23:23:12.138911: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-10-27 23:23:12.138936: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned ab

Best found configuration: {'dropout1_rate': 0.5804777682828415, 'dropout2_rate': 0.14342378718784596, 'lr': 1.2875175408769352e-06, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 442, 'dense1_units': 113}
Working on:  classification rgb 9 5 avg
./modelling_results/rgb_classification_9_5_avg_0  under processing
(32410, 1024) Counter({4.0: 4630, 2.0: 4630, 1.0: 4630, 3.0: 4630, 0.0: 4630, -2.0: 4630, -1.0: 4630})
(7875, 1024) Counter({0.0: 1125, 2.0: 1125, 4.0: 1125, 3.0: 1125, 1.0: 1125, -2.0: 1125, -1.0: 1125})
(4790, 1024) Counter({0.0: 958, 1.0: 958, 2.0: 958, 3.0: 958, 4.0: 958})


/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


Best found configuration: {'dropout1_rate': 0.7227782661331077, 'dropout2_rate': 0.7642971670657839, 'lr': 0.006677802781385853, 'num_dense_layers': 3, 'optimizer': 'SGD', 'start_neurons_units': 193, 'dense1_units': 109, 'dense2_units': 5, 'sgd_momentum': 0.6339122305883391}
Working on:  classification rgb 9 2.5 none
./modelling_results/rgb_classification_9_2.5_none_0  already processed
Working on:  classification rgb 5 5 conc
./modelling_results/rgb_classification_5_5_conc_0  under processing
(24132, 2048) Counter({1.0: 6033, 2.0: 6033, -1.0: 6033, 0.0: 6033})
(4960, 2048) Counter({1.0: 1240, 2.0: 1240, 0.0: 1240, -1.0: 1240})
(3477, 2048) Counter({0.0: 1159, 1.0: 1159, 2.0: 1159})


...

Best found configuration: {'dropout1_rate': 0.7000570107883668, 'dropout2_rate': 0.6711980315608913, 'lr': 8.189116094256225e-06, 'num_dense_layers': 1, 'optimizer': 'SGD', 'start_neurons_units': 47, 'sgd_momentum': 0.41896507763575214}
Working on:  classification twostream 3 5 avg
./modelling_results/twostream_classification_3_5_avg_0  under processing
(22686, 2048) Counter({1.0: 7562, 0.0: 7562, -1.0: 7562})
(4764, 2048) Counter({1.0: 1588, -0.0: 1588, -1.0: 1588})
(5433, 2048) Counter({0.0: 1811, 1.0: 1811, -1.0: 1811})


/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


Best found configuration: {'dropout1_rate': 0.7424510778271874, 'dropout2_rate': 0.8191234982847261, 'lr': 7.876884093633973e-05, 'num_dense_layers': 3, 'optimizer': 'SGD', 'start_neurons_units': 125, 'dense1_units': 22, 'dense2_units': 28, 'sgd_momentum': 0.667157285300489}
Working on:  classification twostream 3 2.5 none
./modelling_results/twostream_classification_3_2.5_none_0  under processing
(45588, 2048) Counter({1.0: 15196, 0.0: 15196, -1.0: 15196})
(9528, 2048) Counter({1.0: 3176, -0.0: 3176, -1.0: 3176})
(10884, 2048) Counter({0.0: 3628, 1.0: 3628, -1.0: 3628})


/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


Best found configuration: {'dropout1_rate': 0.799576254907627, 'dropout2_rate': 0.20614030527795477, 'lr': 1.37790034250251e-05, 'num_dense_layers': 2, 'optimizer': 'SGD', 'start_neurons_units': 161, 'dense1_units': 31, 'sgd_momentum': 0.11269389481070538}


### modeling handcrafted 

In [6]:
prblemTypes = ['classification']
featureTypes = ['handcrafted']
classes = [9,5,3]
# divides = [2.5, 5]
fusionTypes = ['2.5','5','5s']

permutations=[ prblemTypes, featureTypes, classes, fusionTypes]
all_permutations = list(itertools.product(*permutations))
print(len(all_permutations))
for this_permutation in all_permutations:
    (prblemType, featureType, eng_lvls, fusionType) = this_permutation
    classType = 'round_avg_eng_level' if prblemType == 'classification' else 'avg_eng_level'
    
    print('Working on: ',prblemType, featureType, eng_lvls, fusionType)
    
    label_folder = os.path.join(lables_path,'_'.join([classType,'eng_lvl',prblemType,str(eng_lvls),str(fusionType)]))
    feature_folder = os.path.join(features_path,'_'.join([featureType,'features',fusionType]))
        
    #save results like: rgb_classification_9_2.5_none
    result_folder = os.path.join(results_path,'_'.join([featureType,prblemType,str(eng_lvls),fusionType]))

    create_modlling(label_folder,feature_folder,result_folder, prblemType, featureType, eng_lvls)#, divide,fusionType)
    

9
Working on:  classification handcrafted 9 2.5
./modelling_results/handcrafted_classification_9_2.5_0  under processing
(73512, 330) Counter({2.0: 9189, 0.0: 9189, 1.0: 9189, 4.0: 9189, 3.0: 9189, -1.0: 9189, -3.0: 9189, -2.0: 9189})
(14994, 330) Counter({3.0: 2142, 2.0: 2142, 1.0: 2142, -1.0: 2142, 4.0: 2142, 0.0: 2142, -2.0: 2142})
(11538, 330) Counter({0.0: 1923, 1.0: 1923, 2.0: 1923, 3.0: 1923, 4.0: 1923, -1.0: 1923})


2021-11-03 00:46:26.862765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-03 00:46:26.867246: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-03 00:46:26.867941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sharifa/catkin_ws/devel/lib:/usr/local/cuda-11.0/lib64
2021-11-03 00:46:26.867957: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned ab

Best found configuration: {'dropout1_rate': 0.7145244590917286, 'dropout2_rate': 0.039123790592088904, 'lr': 0.005924966666390326, 'num_dense_layers': 1, 'optimizer': 'Adam', 'start_neurons_units': 48}
Working on:  classification handcrafted 9 5
./modelling_results/handcrafted_classification_9_5_0  under processing
(31836, 330) Counter({2.0: 4548, 0.0: 4548, 1.0: 4548, 3.0: 4548, -1.0: 4548, 4.0: 4548, -2.0: 4548})
(7462, 330) Counter({3.0: 1066, 2.0: 1066, 0.0: 1066, 1.0: 1066, 4.0: 1066, -2.0: 1066, -1.0: 1066})
(4735, 330) Counter({0.0: 947, 1.0: 947, 2.0: 947, 3.0: 947, 4.0: 947})


/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


Best found configuration: {'dropout1_rate': 0.75829021658743, 'dropout2_rate': 0.5120882019465703, 'lr': 0.08768105253686996, 'num_dense_layers': 2, 'optimizer': 'Adam', 'start_neurons_units': 60, 'dense1_units': 18}
Working on:  classification handcrafted 9 5s
./modelling_results/handcrafted_classification_9_5s_0  under processing
(31544, 330) Counter({2.0: 3943, 1.0: 3943, 4.0: 3943, 3.0: 3943, 0.0: 3943, -1.0: 3943, -2.0: 3943, -3.0: 3943})
(6440, 330) Counter({3.0: 920, 0.0: 920, 2.0: 920, 4.0: 920, 1.0: 920, -2.0: 920, -1.0: 920})
(4938, 330) Counter({0.0: 823, 1.0: 823, 2.0: 823, 3.0: 823, 4.0: 823, -1.0: 823})


...

Best found configuration: {'dropout1_rate': 0.5057042097525165, 'dropout2_rate': 0.8051212700878719, 'lr': 8.928816067621497e-05, 'num_dense_layers': 2, 'optimizer': 'SGD', 'start_neurons_units': 425, 'dense1_units': 26, 'sgd_momentum': 0.8958535927391047}
Working on:  classification handcrafted 3 5
./modelling_results/handcrafted_classification_3_5_0  under processing
(22254, 330) Counter({1.0: 7418, 0.0: 7418, -1.0: 7418})
(4497, 330) Counter({0.0: 1499, 1.0: 1499, -1.0: 1499})
(5355, 330) Counter({0.0: 1785, 1.0: 1785, -1.0: 1785})


/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


Best found configuration: {'dropout1_rate': 0.00921622759991524, 'dropout2_rate': 0.6483605999222419, 'lr': 0.05299509987301488, 'num_dense_layers': 1, 'optimizer': 'SGD', 'start_neurons_units': 260, 'sgd_momentum': 0.16927825739526883}
Working on:  classification handcrafted 3 5s
./modelling_results/handcrafted_classification_3_5s_0  under processing
(19122, 330) Counter({1.0: 6374, 0.0: 6374, -1.0: 6374})
(3849, 330) Counter({1.0: 1283, 0.0: 1283, -1.0: 1283})
(4578, 330) Counter({0.0: 1526, 1.0: 1526, -1.0: 1526})


/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
/home/sharifa/engagement_modeling/eng_env/lib/python3.8/site-packages/keras/backend.py:4846: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


Best found configuration: {'dropout1_rate': 0.554223447257964, 'dropout2_rate': 0.827890666633727, 'lr': 0.00017767890302388274, 'num_dense_layers': 1, 'optimizer': 'SGD', 'start_neurons_units': 187, 'sgd_momentum': 0.1391350005625941}
